## Extracting location

In [12]:
from urllib.request import Request, urlopen
from  bs4 import BeautifulSoup
import pandas as pd
import csv
import json
import os
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time 
import json

database_credential = json.load(open('credentials.json'))
connection = create_engine(database_credential['db'])

In [2]:
chaldalURL = ['https://chaldal.com/fresh-fruit', 'https://chaldal.com/fresh-vegetable', 'https://chaldal.com/cheeses']

def makeRequest(url):
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})
    return urlopen(req).read()
    

In [3]:
path = "https://www.pizzahutbd.com/store-search?location="
all_data = makeRequest(path)
all_data = BeautifulSoup(all_data, 'html.parser')
stores = all_data.find_all("div", {"class": "store_card"})

In [4]:
store_list = []

for i in range(len(stores)):
    store_name = stores[i].find_all('span', {'class' : 'storename'})[0].text.replace("Pizza Hut ","")
    store_name = store_name.replace("Delivery ","")
    store_name = store_name.replace("FCD ","")
    store_list.append(store_name)

In [5]:
print(store_list)

['Gulshan', 'RM Center', 'Dhanmondi', 'Dhanmondi', 'Baily Road', 'Uttara', 'Uttara', 'Bhatara', 'Banani', 'Mirpur', 'Sony', 'Wari', 'Savar', 'Cox Bazar', 'Chittagong', 'Sylhet', 'Jamuna', 'Adabor', 'Banasree', 'Shewrapara', 'Gulshan 2']


In [6]:
store_data = pd.DataFrame({'location': store_list})
store_data['restaurant'] = None
store_data['restaurant'] = "Pizza Hut"

store_data.to_sql(con=connection, name = 'location_data', if_exists='append', index=False)

## Extracting Product name and price

In [8]:
# Function to extract name and price from responses
def extract_name_price(num_products, option = 'default'):
    
    total_products = len(list(set(num_products.text.split("\n"))))

    for i in range(total_products):
        try:
            product_name = num_products.find_element("xpath", f'//*[@id="productList"]/div[{i}]/form/div/div[2]/div[1]').text
            product_name_list.append(product_name)
            
            if option == 'default':
                product_price = num_products.find_element("xpath", f'//*[@id="productList"]/div[{i}]/form/div/div[2]/div[5]/button/span').text
                product_price_list.append(product_price)
            elif option == 'Appetisers':
                product_price = num_products.find_element("xpath", f'//*[@id="productList"]/div[{i}]/form/div/div[2]/div[4]/button/span').text
                product_price_list.append(product_price)
                
        except:
            pass

In [9]:
url = "https://www.pizzahutbd.com/"

product_name_list = []
product_price_list = []

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

# Click on search button
search = driver.find_element("xpath", '//*[@id="delivery"]/div/div/div/span')
search.click()
time.sleep(3)

########### All Pizza ###########
all_pizza =  driver.find_element("id", 'productList')

extract_name_price(all_pizza)
time.sleep(3)

########### Cheezy bit Pizza ###########
cheesy_bites_path = '//*[@id="pnProductNavContents"]/a[2]'
cheesy_bites = driver.find_element("xpath", cheesy_bites_path)
cheesy_bites.click()

Cheezy =  driver.find_element("id", 'productList')

extract_name_price(Cheezy)
time.sleep(3)

########### SAUSAGE Pizza ###########
sausages_path = '//*[@id="pnProductNavContents"]/a[3]'
sausages = driver.find_element("xpath", sausages_path)
sausages.click()

sausages =  driver.find_element("id", 'productList')

extract_name_price(sausages)
time.sleep(3)

########### thin Pizza ###########
thin_path = '//*[@id="pnProductNavContents"]/a[4]'
thin = driver.find_element("xpath", thin_path)
thin.click()

thin =  driver.find_element("id", 'productList')
extract_name_price(thin)
time.sleep(3)

########### pasta ###########
pasta_path = '//*[@id="navbar"]/ul/li[2]/a'
pasta = driver.find_element("xpath", pasta_path)
pasta.click()

pasta =  driver.find_element("id", 'productList')
extract_name_price(pasta)
time.sleep(3)

########### appetisers ###########
appetisers_path = '//*[@id="navbar"]/ul/li[3]/a'
appetisers = driver.find_element("xpath", appetisers_path)
appetisers.click()

appetisers =  driver.find_element("id", 'productList')
extract_name_price(appetisers,'Appetisers')
time.sleep(3)

########### drinks ###########
drinks_path = '//*[@id="navbar"]/ul/li[5]/a'
drinks = driver.find_element("xpath", drinks_path)
drinks.click()

drinks =  driver.find_element("id", 'productList')
extract_name_price(drinks,'Appetisers')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\my pc\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


## Saving to the dataframe

In [11]:
name_price = pd.DataFrame({'name': product_name_list, 'price': product_price_list})

name_price['restaurant'] = None
name_price['restaurant'] = "Pizza Hut"

name_price.to_csv("./data/2_pizzahut.csv", index=False)
name_price.to_sql(con=connection, name = 'pizzahut', if_exists='replace', index=False)

name_price

,name,price,restaurant
0,Classic Margherita,179,Pizza Hut
1,Veggie Lovers,199,Pizza Hut
2,Classic Beef,269,Pizza Hut
3,Ultimate Hot Chicken,269,Pizza Hut
4,Classic Chicken Sausage,269,Pizza Hut
5,Chicken Exotica,329,Pizza Hut
6,Spicy Beef.,329,Pizza Hut
7,Spicy Chicken.,379,Pizza Hut
8,Beef Lovers.,379,Pizza Hut
9,Veggie Supreme,329,Pizza Hut
